# Introduction
I've seen that there are quite some issues with the nucleotide metabolism that seem to be the root of a lot of problems we are having in the model. Here I will try to systematically tackle this issue, going from the 'roots': Uridine, Guanine, etc. And hope that we somehow can clarify what is happening here. 

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction, Metabolite

In [2]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [3]:
model_e_coli = cameo.load_model ('iML1515')

In [4]:
model_b_sub = cameo.load_model('iYO844')

### Uridine to ump node
Here I start looking into the NTP metabolism, starting from the 'beginning' Uridine.

In [5]:
rct_id = []
rct_rct =[]
for rct in model.metabolites.uri_c.reactions:
    if rct in model.metabolites.ump_c.reactions:
        rct_id.append(rct.id)
        rct_rct.append(rct.reaction)
    else: continue
rct_df = pd.DataFrame({'ID':rct_id, 'reaction':rct_rct})
rct_df

,ID,reaction
0,URIK3,itp_c + uri_c <=> idp_c + ump_c
1,UTUP,uri_c + utp_c <=> udp_c + ump_c
2,DGTUP,dgtp_c + uri_c --> dgdp_c + ump_c
3,DTTUP,dttp_c + uri_c <=> dtdp_c + ump_c
4,DUTUP,dutp_c + uri_c <=> dudp_c + ump_c
5,URIK2,gtp_c + uri_c <=> gdp_c + ump_c
6,DATUP,datp_c + uri_c <=> dadp_c + ump_c
7,DCTUP,dctp_c + uri_c <=> dcdp_c + ump_c
8,URIK1,atp_c + uri_c --> adp_c + ump_c
9,NTD2,h2o_c + ump_c --> h_c + pi_c + uri_c


There are really many reactions that have both ump and uri,I think this is interesting and warrents some more investigation. 
- Th ecoli model only has the reactions UIK1, URIK2 and NTD2. B. sub only has NTD2, URIK1, URIK2 and URIK3. So I would expect that a lot of these reactions should be removed and are maybe somewhere the cause of this nucleotide problem I am having.

I will look into the other reactions: is it possible to perform the reactions as the DXTUP reactions all contain?

In [6]:
#URIK2 and URIK3 should be irreversible
model.reactions.URIK2.bounds = (0,1000)
model.reactions.URIK3.bounds = (0,1000)

It seems these reactions refer to the nucleotide kinases, e.g. EC 2.7.1.48 - uridine kinase. These however work with the NTPs, not the dNTPs as are included here in the reaction. This enzyme is known to be promiscuous for the other NTPs, and this is already reflected by the URIK1/2/3 reactions in the model. the conversion with the dNTP variant seems to be a mistake in Kegg. the dNTPs should be coupled to the de-oxyuridine (duri_c) metabolite. 

So: I will remove these reactions and only leave the URIK1/2/3 and NTD2 reactions in place and see what it does to the model. As it doesnt really influence the model much, so I will remove them.

UTUP is a bit different than the other 5 that will be removed: it resembles the URIK reactions more. So i will leave it in the model for now, just make it irreversible like the other reactions it is similar to. 


In [9]:
remove = ['DGTUP', 'DUTUP', 'DCTUP', 'DTTUP', 'DATUP']
for rct in model.reactions:
    if rct.id in remove:
        model.remove_reactions(rct)
    else: 
        continue

In [8]:
model.reactions.UTUP.bounds = (0,1000)

In [11]:
#save& commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

### Guanine to guanosine to gmp node
PUMP3 and PUNP4 seem to be correct and should be present. Here I've fixed it so that it matches what it should be, but still the flux is a bit unpredicted. This can come from other problems in the biosynthesis though, so will be left for now.

In [29]:
model.reactions.GMP5N.id = 'NTD9'

In [36]:
model.reactions.GPAR.id = 'GUAPRT'

In [37]:
model.reactions.GUAPRT.bounds = (0,1000)

In [72]:
#save& commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

### Cytosine & cytodine
Seems cytosine is only involved in one reaction in geobacillus. That means currently it is a blocked reaction and will never carry any flux...

keep rcts: CYTCD, PYNP1, NTD4, CYTDK2, CYTDK1

The other reactions are similar to what we saw with the uridine case, so all these reactions should be removed and we will later come by the de-oxycyrosine anyway. 

The ITCY and UCTY reaction I will leave, as they resemble teh CYTDK reactions just with other nucleotide triphosphates. I just will make them irreversible, as that makes more sense. 

In [101]:
model.reactions.CYTDK2.bounds= (0,1000)

In [109]:
model.reactions.PYNP1.bounds = (0,1000)  

In [116]:
remove = ['DTTGY', 'DUTCP', 'DCTCP','DATCY', 'DGTCY']
for rct in model.reactions:
    if rct.id in remove:
        model.remove_reactions(rct)
    else: 
        continue

In [122]:
model.reactions.ITCY.bounds = (0,1000)
model.reactions.UTCY.bounds = (0,1000)

In [124]:
#save & commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

### Adenine
from adenine (ade_c) to adenosine (adn_c) to amp

In [141]:
model.reactions.ADPT.bounds = (0,1000)

### Xanthine
This strain is known to also make XMP and IMP, and so i will check those now. 

For xanthine it gets converted to xanthosine and then into XMP


In [154]:
model.reactions.XPPT.bounds = (0,1000)

## Inosine
seems to be correct.

In [162]:
#save& commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

In [34]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

### deoxy-cytidine & deoxy-uridine
Seems to be correct

__Deoxy-guanosine__ Needs to change one set of bounds

In [35]:
model.reactions.NTD8.bounds = (0,1000)

__Hypoxanthine__ Need to change the bounds

In [36]:
model.reactions.HXPRT.bounds = (-1000,0)

__deoxy-inosine__ This seems to be a dead-end metabolite here, as it is only present in one reaction. This is because dIMP,dIDP and dITP are not in the model. For now I will leave this unless i later find an indication as to why this should be added into the model.

__Deoxy-adenosine__ 

We have both dad_2_c and dad_5_c metabolites.Here I will also quickly look into that the dad_5_c metabolite node is correct. I don't really see anything wrong with it straight away so I will leave it as it is and maybe come back to it later.

In [37]:
model.reactions.NTD6.bounds = (0,1000)

__dTMP__
Looking into the forming of dTMP from thymidine, the node seems correct.

In [39]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')